<h1> word2vec Dataset 2</h1>

In [1]:
# import libraries

import os
import sys
import numpy as np
import pandas as pd
import matplotlib as plt
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
from transformers import TrainingArguments
from sklearn.metrics import f1_score
import json
import wandb

# import warnings
# warnings.filterwarnings(action='ignore')

In [2]:
mps_device = torch.device("mps")
# define helper functions

def argmax(vec):

    """ 
    argmax as the name suggests finds the value 
    of the argument that gives the maximum value
    of a target function
    """

    # return max value of all elements in the input tensor
    _, idx = torch.max(vec, 1) 
    return idx.item()

def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long, device=mps_device)

# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]

    # view returns a new tensor w/ a different shape
    # maintaining the number of elements

    # expand simply expands a tensor to a larger size
    # say ([1], [2], [3]).expand(3,3) would give
    # ([1,1,1],
    #  [2,2,2],
    #  [3,3,3])

    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    
    return max_score + torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

In [3]:
mps_device = torch.device("mps")

class BiLSTM_CRF(nn.Module):
    #BILSTM CRF is a subclass inheriting from the (nn.Module) superclass

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim, word_embeds):
        super(BiLSTM_CRF, self).__init__() #call the init method of the superclass (nn.Module)

        START_TAG = "<START>"
        STOP_TAG = "<STOP>"
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        # lookup table that stores embeddings
        # self.word_embeds = nn.Embedding(vocab_size, embedding_dim).to(mps_device)
        self.word_embeds = word_embeds

        #define the lstm
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True).to(mps_device)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size).to(mps_device)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size)).to(mps_device)

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2).to(mps_device),
                torch.randn(2, 1, self.hidden_dim // 2).to(mps_device))

    def _forward_alg(self, feats):

        START_TAG = "<START>"
        STOP_TAG = "<STOP>"
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(mps_device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)

        return lstm_feats

    def _score_sentence(self, feats, tags):

        START_TAG = "<START>"
        STOP_TAG = "<STOP>"

        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(mps_device)
        # tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])

        tags_tensor = torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long, device=mps_device)
        tags_tensor = torch.cat([tags_tensor, tags.to(mps_device)])
        
        for i, feat in enumerate(feats):

            score = score + self.transitions[tags_tensor[i + 1], tags_tensor[i]] + feat[tags_tensor[i + 1]]

        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags_tensor[-1]]
        return score

    def _viterbi_decode(self, feats):

        START_TAG = "<START>"
        STOP_TAG = "<STOP>"
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000., device=mps_device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
            
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [6]:
mps_device = torch.device("mps")

START_TAG = "<START>"
STOP_TAG = "<STOP>"

# Load training data
training_data = []

# getting training sentence tokens
training_file_path = "/Users/mo/Desktop/repos/nlp_a2/task_2/dataset/train_bio.json"

with open(training_file_path, "r") as f:
    data = json.load(f)
    for entry in data.values():
        sentence = entry["text"].split()
        tags = entry["labels"]
        training_data.append((sentence, tags))

word_to_ix = {}
for sentence, tags in training_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

tag_to_ix = {
    "B": 0,
    "I": 1,
    "O": 2,
    START_TAG: 3,
    STOP_TAG: 4
}

# model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM).to(mps_device)
# optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)

# # Check predictions before training
# with torch.no_grad():
#     precheck_sent = prepare_sequence(training_data[0][0], word_to_ix).to(mps_device)
#     precheck_tags = torch.tensor([tag_to_ix[t] for t in training_data[0][1]], dtype=torch.long).to(mps_device)
#     print(model(precheck_sent))

# for epoch in range(5): 
    
#     print(f"epoch num: {epoch+1}")

#     sentence_count = 0
    
#     for sentence, tags in training_data:

#         sentence_count += 1
#         # Step 1. Remember that Pytorch accumulates gradients.
#         # We need to clear them out before each instance
#         model.zero_grad()

#         # Step 2. Get our inputs ready for the network, that is,
#         # turn them into Tensors of word indices.
#         sentence_in = prepare_sequence(sentence, word_to_ix).to(mps_device)
#         targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long).to(mps_device)

#         # Step 3. Run our forward pass.
#         loss = model.neg_log_likelihood(sentence_in, targets)

#         # Step 4. Compute the loss, gradients, and update the parameters by
#         # calling optimizer.step()
#         loss.backward()
#         optimizer.step()

#         if (sentence_count % 1000 == 0):
#             print(f"{sentence_count}/{len(training_data)} sentences processed")

# # Check predictions after training
# with torch.no_grad():
#     precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
#     print(model(precheck_sent))

In [7]:
# generate word2vec embeddings for the words in the training data
from gensim.models import KeyedVectors

EMBEDDING_DIM = 300
HIDDEN_DIM = 4

# load word2vec model
word2vec_model = KeyedVectors.load_word2vec_format('/Users/mo/Downloads/GoogleNews-vectors-negative300.bin', binary=True)

# init torch embedding layer
word_embeds = nn.Embedding(len(word_to_ix), EMBEDDING_DIM).to(mps_device)

# load word2vec embeddings of size 300 into torch embedding layer
for word, idx in word_to_ix.items():
    if word in word2vec_model:
        word_embeds.weight.data[idx] = torch.tensor(word2vec_model[word], device=mps_device)

In [8]:
# load the model

model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, word_embeds).to(mps_device)
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)

In [9]:
#check predictions before training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix).to(mps_device)
    precheck_tags = torch.tensor([tag_to_ix[t] for t in training_data[0][1]], dtype=torch.long).to(mps_device)
    print(model(precheck_sent))

(tensor(18.8682, device='mps:0'), [1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 1, 2, 1, 2])


In [ ]:
train_loader, val_loader = get_data(tokenizer)
model, optimizer, criterion = get_model(model)

# Initialize W&B
wandb.login(key="7ef2e84866a68a6cd33c90b1fa55c8cf8ab2d6e7", relogin=True)
wandb.init(project="nlp_a2", name="BiLSTM_CRF_w2v_t2")
wandb.watch(model)

# Train and evaluate the model
num_epochs = 10
best_val_f1 = 0
best_val_loss = 100
for epoch in tqdm(range(num_epochs), desc="Epochs"):
    model.train()
    train_loss = 0
    train_f1 = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.view(-1, 3), labels.view(-1))
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

        # Calculate F1-score
        predicted = outputs.argmax(dim=2).cpu().numpy()
        true = labels.cpu().numpy()
        train_f1 += f1_score(true.flatten(), predicted.flatten(), average="macro")

    model.eval()
    val_loss = 0
    val_f1 = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs.view(-1, 3), labels.view(-1))
            val_loss += loss.item()

            # Calculate F1-score
            predicted = outputs.argmax(dim=2).cpu().numpy()
            true = labels.cpu().numpy()
            val_f1 += f1_score(true.flatten(), predicted.flatten(), average="macro")

    # Log metrics to W&B
    train_loss /= len(train_loader)
    val_loss /= len(val_loader)
    train_f1 /= len(train_loader)
    val_f1 /= len(val_loader)

    wandb.log(
        {
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "val_loss": val_loss,
            "train_f1": train_f1,
            "val_f1": val_f1,
        }
    )

    # Save the best model checkpoint
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save(model.state_dict(), run_name + ".pt")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1

    # Check if training should be stopped
    if epochs_without_improvement >= 3:
        print(f"Stopping early at epoch {epoch+1} due to no improvement.")
        break

    print(
        f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss}, Val Loss: {val_loss}, Train F1: {train_f1}, Val F1: {val_f1}"
    )

wandb.finish()